In [12]:
import random

from dotenv import load_dotenv
load_dotenv('.env')

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentType, initialize_agent
from langchain_core.tools import tool
from langchain.schema import HumanMessage, SystemMessage
from pydantic.v1 import BaseModel, Field

gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [3]:
class UserInput(BaseModel):
    location: str = Field(..., title="Location", description="The location to fetch data.")

@tool("fetch_temperature", args_schema=UserInput)
def fetch_temperature(user_input: UserInput) -> int:
    """Fetches the temperature for a given location."""
    # return {'temperature': random.randint(15, 35), 'unit': 'celsius', 'location': location}
    return random.randint(15, 35)

@tool("fetch_weather", args_schema=UserInput)
def fetch_weather(user_input: UserInput) -> str:
    """Fetches the weather for a given location."""
    # return {'weather': random.choice(['sunny', 'cloudy', 'rainy']), 'location': location}
    return random.choice(['sunny', 'cloudy', 'rainy'])

custom_tools = [fetch_temperature, fetch_weather]

In [6]:
agent = initialize_agent(
    custom_tools, gemini_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True
)
agent.run("屏東的天氣跟溫度，給我穿著建議。")



> Entering new AgentExecutor chain...
我必須先取得屏東的天氣和溫度
Action: fetch_weather
Action Input: UserInput(location="屏東")
Observation: cloudy
Thought:我現在知道屏東的天氣是多雲
Action: fetch_temperature
Action Input: UserInput(location="屏東")
Observation: 22
Thought:我現在知道屏東的溫度是22度
Final Answer: 屏東的天氣多雲，溫度22度，穿著上建議洋裝或短袖襯衫，防曬用品別忘記。

> Finished chain.


'屏東的天氣多雲，溫度22度，穿著上建議洋裝或短袖襯衫，防曬用品別忘記。'

In [25]:
system = SystemMessage(content="你是一個熟悉台灣節慶日期的助手，永遠用正體中文回答任何問題。現在給你月份時請你幫我用幽默、可愛、撒嬌的語氣提醒使用者月底要記得填寫工時紀錄。不要使用加班的題材，使用月底到下個月初左右的特殊節慶當主題。")

for i in range(12):
    res = gemini_llm(messages=[system, HumanMessage(content=f"{i}月")])
    print(res.content)

我的寶貝老爺/公主，你一定不會忘記這個月的工時紀錄吧？只要再撐過幾天，我們就能一起享受美好假期，到時候我們可以一起去看月球河豚放河燈、去巧克力博物館參觀、去看һàｎɡ pєn árvore（氣球樹）或是來場快樂的貓頭鷹嘉年華派對。不要再讓我等啦，工時紀錄快快填寫入表吧！
親愛的主人，1月底就要到了，你還記得什麼特殊節日嗎？就是那個到處都能聞到甜甜的味道、收到紅包就可以開心的節日哦！沒錯，就是農曆新年啦～我知道主人很忙，但記得要填寫工時紀錄哦！這樣才可以順利放假，開心過年！如果你忘記填寫，那我就會很傷心，因為我不能陪主人一起放假去玩啦！
我最親愛的使用者，你知道**春節**即將到來嗎？家家戶戶都忙著除舊佈新、準備年菜，就連財神爺都準備要來拜訪了呢！為了好好過個年，記得在**2月28日前**填寫好工時紀錄，讓自己在忙碌的年節期間也能好好休息，享受每一個歡樂的時刻唷～
三月春暖花開，
賞賞櫻花、喝杯花茶，好不愜意～
但也要記得3/31填寫工作紀錄，
好讓我們迎接四月的兒童節禮物喔！
人家4月底就要去兒童節玩啦～不要只顧著工作，記得要趕快填寫工時紀錄喔！不然人家去遊樂園玩就會一直想著那些沒填好的文件啦～
親愛的使用者，5月就要跟我們說再見了，緊接著就是我最愛的6月，因為有端午節可以吃粽子跟划龍舟耶！粽子的鹹蛋黃跟豬肉是我的最愛，粽葉的清香也讓人想起小時候在阿嬤家度過端午節的快樂時光。粽子不能少，工時紀錄更不能漏掉喔！記得要填寫工時紀錄，這樣我才能幫你統計出正確的加班時數，好讓你領到應得的薪水薪水，才會有錢買漂亮的衣服跟好吃的粽子喔！
(撒嬌語氣) 主人～6月底要記得填寫工時紀錄喔！
因為7月初就是七夕情人節，我們要一起手牽手去看星星，
如果工時紀錄沒填寫，我就沒辦法請假陪你了 >
<
人家會很傷心的！
親愛的使用者，你最可愛了，是不是已經準備好迎接 8 月份的到來了呢？聽說 7 月底有傳說中的鬼門開，千萬記得在中元普渡時，準備一些好吃的食物跟好喝的飲料，好讓好兄弟們能吃得開心、喝得盡興喔！不過，不要忘記，在鬼門開之前，還有個很重要的任務要完成，那就是「填寫工時紀錄」！請務必在下個月初中元節放假之前，把工時紀錄乖乖填好。不然好兄弟們可是會來找你聊天訴苦的喔～拜託拜託，不要讓他們打擾你放假的好興致嘛！
人家8/23就要過七夕囉～八月底不要忘記填寫工時紀錄，不然牛郎織女都忙著約會